In [1]:
from pymysql.cursors import DictCursorMixin, Cursor
import pymysql
import pandas as pd

In [2]:
connection = pymysql.connect(host='oselimw0206v.int.msdp.ericsson.se',
                             user='tarsys',
                             password='tarsys',
                             db='operdb_smart_ph_1',  #db1474364937925 db1477556785065
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

OperationalError: (2003, "Can't connect to MySQL server on 'oselimw0206v.int.msdp.ericsson.se' ([Errno 11004] getaddrinfo failed)")

In [21]:
sql="""
SELECT 
`earfcndl` as earfcndl,
`bandName`,
bandNumber,
COUNT(DISTINCT `enodeB`) AS noNodes,
COUNT(`EUtranCellFDDId`) AS noEutranCells
FROM
(
SELECT
*
FROM 
e4g_sectorlocation) t
GROUP BY `earfcndl`
;
"""

In [22]:
with connection.cursor() as cursor:
    cursor.execute(sql)
try:
    connection.commit()
except:
    print('Check Value error with the query')
result = cursor.fetchall()

In [23]:
writer = pd.ExcelWriter('simplesQueries.xlsx')
pd.DataFrame(result).to_excel(writer,'noCellsPerEarfcndl')
pd.DataFrame(result)

,bandName,bandNumber,earfcndl,noEutranCells,noNodes
0,2100,1.0,150.0,1275,433
1,1800+,3.0,1350.0,1094,369
2,1800+,3.0,1375.0,3,1
3,1800+,3.0,1750.0,632,212
4,1800+,3.0,1775.0,3,1
5,700 APT,28.0,9475.0,51,17
6,700 APT,28.0,9485.0,663,222
7,TD 2300,40.0,38925.0,3,1
8,error,0.0,41080.0,3,1


In [24]:
sql="""
SELECT 
`bandName`,
dlChannelBandwidth,
bandNumber,
COUNT( DISTINCT `EUtranCellFDDId`) AS noEutranCells
FROM
(
        SELECT
        *
        FROM 
        e4g_sectorlocation
) t
GROUP BY `bandName`,dlChannelBandwidth
ORDER BY 4 DESC
;
"""

In [25]:
with connection.cursor() as cursor:
    cursor.execute(sql)
try:
    connection.commit()
except:
    print('Check Value error with the query')
result = cursor.fetchall()

In [26]:
pd.DataFrame(result)

,bandName,bandNumber,dlChannelBandwidth,noEutranCells
0,2100,1.0,10000.0,1272
1,1800+,3.0,15000.0,993
2,1800+,3.0,10000.0,736
3,700 APT,28.0,15000.0,654
4,700 APT,28.0,20000.0,48
5,700 APT,28.0,10000.0,12
6,1800+,3.0,5000.0,3
7,error,0.0,20000.0,3
8,TD 2300,40.0,20000.0,3
9,2100,1.0,5000.0,3


In [27]:

pd.DataFrame(result).to_excel(writer,'noCellsPerBandBW')
writer.save()